# Test dataset loaders

In [ ]:
import numpy as np

## mini imagenet

### datamanager

In [1]:
import os.path as osp
from PIL import Image

from torch.utils.data import Dataset
from torchvision import transforms

ROOT_PATH = './materials/'
IMG_PATH = 

setname = 'train'

csv_path = osp.join(ROOT_PATH, setname + '.csv')
lines = [x.strip() for x in open(csv_path, 'r').readlines()][1:]
IMG_PATH = "./datasets/mini-imagenet/"
data = []
label = []
lb = -1

wnids = []

for l in lines:
    name, wnid = l.split(',')
    path = osp.join(IMG_PATH, 'images', name)
    if wnid not in wnids:
        wnids.append(wnid)
        lb += 1
    data.append(path)
    label.append(lb)

data = data
label = label

In [10]:
np.unique(label, return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63]),
 array([600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600,
        600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600,
        600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600,
        600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600,
        600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 600]))

### train

In [11]:
import os.path as osp
import os

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from datamanager.miniimagenet_aug import MiniImageNet
from samplers import CategoriesSampler
from convnet import ConvNet
from algorithm.subspace_projection import Subspace_Projection
from utils import pprint, set_gpu, Averager, Timer, count_acc, flip

In [16]:
shot = 1
query = 15
train_way = 5
test_way = 5
save_path = './save/subspace-5w5sdiscriminative'
saved_model_path = './save/subspace-5w5sdiscriminative/max-acc.pth'
data_path = './datasets/mini-imagenet'
gpu = 0
lamb = 0.03

In [13]:
subspace_dim = shot - 1

In [14]:
trainset = MiniImageNet('train', data_path)
train_sampler = CategoriesSampler(trainset.label, 100,
                                  train_way, shot + query)
train_loader = DataLoader(dataset=trainset, batch_sampler=train_sampler,
                          num_workers=8, pin_memory=True)

model = ConvNet().cuda()
model.load_state_dict(torch.load(saved_model_path))

projection_pro = Subspace_Projection(subspace_dim)

In [ ]:
for i, batch in enumerate(train_loader, 1):
    p = shot * train_way
    qq = p + query * train_way
    data_shot, data_query = data[:p], data[p:qq]
    print(f"p: {p}, qq: {qq}, ")
    print(f"batch[1]: {batch[1]}")
    print(f"data_shot: {data_shot}")
    # print(f"data_query: {data_query}")
    if i > 5:
        break
    